In [ ]:
!pip install langchain
!pip install langchain_community
!pip install openai
!pip install tiktoken

In [43]:
%%writefile .env
OPENAI_API_KEY='sk-7INmRu6iHEope6cJCa462b608aFb4eE281Df64F4E32e95F9'
OPENAI_BASE_URL='https://threefive.gpt7.link/v1'  # 有些国内的代理商需要加v1，有些不需要

ZHIPUAI_API_KEY='68604cc3e382eba3aba03daca6f4f717.fteCwuYKtXWpexN6'
DASHSCOPE_API_KEY='sk-d4c3f773af5d442c97eb7d9e40479c55'

Overwriting .env


In [44]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv(),override=True)

import os
print(os.environ['OPENAI_BASE_URL'],os.environ['OPENAI_API_KEY'])

https://threefive.gpt7.link/v1 sk-7INmRu6iHEope6cJCa462b608aFb4eE281Df64F4E32e95F9


## 使用通义千问

In [45]:
%%writefile qwen.env
## Qwen base_url
OPENAI_BASE_URL='https://dashscope.aliyuncs.com/compatible-mode/v1'
DASHSCOPE_API_KEY='sk-d4c3f773af5d442c97eb7d9e40479c55'
OPENAI_API_KEY=${DASHSCOPE_API_KEY}

Writing qwen.env


In [46]:
## 使用qwen key
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv('qwen.env',override=True)

import os
print(os.environ['OPENAI_BASE_URL'],os.environ['OPENAI_API_KEY'])

https://dashscope.aliyuncs.com/compatible-mode/v1 sk-d4c3f773af5d442c97eb7d9e40479c55


## langchain.chat_models
### 函数ChatOpenAI参数
- model
- temperature
- max_tokens
- stop
- api_key
- base_url

### client类
- 输入为list,list内为一个或多个message对象
- 输出为AIMessage对象

In [72]:
from langchain.chat_models import ChatOpenAI
from langchain_core.messages import HumanMessage,SystemMessage


## ChatOpenAI使用方法
chat = ChatOpenAI(
    model='qwen2-7b-instruct',
    temperature=0.5,
)
print(type(chat))

## ChatOpenAI.call输入格式
prompt='什么是gpt?'
message=[
    SystemMessage('你是一名人工智能专家。'),
    HumanMessage(prompt)
]
print(message)

output=chat(message)
print(type(output))
print(output.content)


<class 'langchain_community.chat_models.openai.ChatOpenAI'>
[SystemMessage(content='你是一名人工智能专家。'), HumanMessage(content='什么是gpt?')]
<class 'langchain_core.messages.ai.AIMessage'>
GPT，全称为Generative Pre-trained Transformer（生成预训练变换器），是由OpenAI在2018年发布的一种大规模语言模型。它是一种基于Transformer架构的深度学习模型，使用了自回归的方法来生成文本。

GPT的主要特点包括：

1. **预训练**：GPT首先在大量的文本数据上进行无监督学习，生成一个通用的语言表示。这个过程被称为预训练，它使得模型能够捕捉到语言的普遍结构和规律。

2. **双向上下文理解**：与传统的循环神经网络（RNN）相比，Transformer模型能够同时处理输入序列的前后文信息，从而更好地理解文本的语义。

3. **自回归生成**：在生成文本时，GPT逐个预测每个词，考虑到之前所有词的影响，这使得生成的文本更加连贯和有意义。

4. **微调**：预训练的GPT模型可以进一步通过有监督的方式进行微调，以适应特定的任务，如问答、文本分类或对话生成等。

GPT系列模型随着时间的推移不断发展，包括GPT-2、GPT-3等版本，它们分别具有更大的参数量和更强大的生成能力。这些模型在各种自然语言处理任务中都取得了显著的成果，并且在研究和开发领域中被广泛使用。


## langchain_core.messages
### 主要的message对象
- SystemMessage
- HumanMessage
- AIMessage
- ToolMessage

### 主要方法
- trim_messages：裁剪输入长度

In [82]:
from langchain_core.messages import (
    AIMessage,
    HumanMessage,
    SystemMessage,
    trim_messages,
)

messages = [
    SystemMessage("you're a good assistant, you always respond with a joke."),
    HumanMessage("i wonder why it's called langchain"),
    AIMessage(
        'Well, I guess they thought "WordRope" and "SentenceString" just didn\'t have the same ring to it!'
    ),
    HumanMessage("and who is harrison chasing anyways"),
    AIMessage(
        "Hmmm let me think.\n\nWhy, he's probably chasing after the last cup of coffee in the office!"
    ),
    HumanMessage("what do you call a speechless parrot"),
]

## 裁剪输入
model_name='gpt-4o'
try:
    trim_messages(
        messages,
        max_tokens=45,
        strategy="last",
        token_counter=ChatOpenAI(model=model_name),
    )
except:
    print(f'模型{model_name}未实现裁剪所需函数！')


## langchain_core.prompts
### PromptTemplate
使用提示模版，可以让我们更为方便地重复使用设计好的提示.
- PromptTemplate
- ChatPromptTemplate
#### 使用方法
先构造一个模版字符串，再调用PromptTemplate.from_template

#### 提示模版使用方法
- template.format_messages
- template.invoke
输入格式见代码。



In [83]:
from langchain_core.prompts import ChatPromptTemplate
from pprint import pprint
# 首先，构造一个提示模版字符串：`template_string`

template_string = """把由三个反引号分隔的文本\
翻译成一种{style}风格。\
文本: ```{text}```
"""

## 然后，我们调用`ChatPromptTemplatee.from_template()`函数将上面的提示模版字符`template_string`转换为提示模版`prompt_template`

prompt_template = ChatPromptTemplate.from_template(template_string)


print(prompt_template.messages)


[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['style', 'text'], template='把由三个反引号分隔的文本翻译成一种{style}风格。文本: ```{text}```\n'))]


In [94]:
customer_style = """正式普通话 \
用一个平静、尊敬的语气
"""

customer_email = """
嗯呐，我现在可是火冒三丈，我那个搅拌机盖子竟然飞了出去，把我厨房的墙壁都溅上了果汁！
更糟糕的是，保修条款可不包括清理我厨房的费用。
伙计，赶紧给我过来！
"""

## 使用提示模版方法1
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email
)
print("客户消息类型:",type(customer_messages),"\n")

## 使用提示模版方法2
## 注意要调用to_message转成list类型
customer_messages = prompt_template.invoke(
    {
        'style':customer_style,
        'text':customer_email
    }
)#.to_message()

# 打印客户消息类型
print("客户消息类型:",type(customer_messages),"\n")
print("客户消息类型:",type(customer_messages.to_messages()),"\n")
print("客户消息: ", customer_messages,"\n")

customer_response = chat(customer_messages.to_messages())
print('输出：')
print(customer_response.content)


客户消息类型: <class 'list'> 

客户消息类型: <class 'langchain_core.prompt_values.ChatPromptValue'> 

客户消息类型: <class 'list'> 

客户消息:  messages=[HumanMessage(content='把由三个反引号分隔的文本翻译成一种正式普通话 用一个平静、尊敬的语气\n风格。文本: ```\n嗯呐，我现在可是火冒三丈，我那个搅拌机盖子竟然飞了出去，把我厨房的墙壁都溅上了果汁！\n更糟糕的是，保修条款可不包括清理我厨房的费用。\n伙计，赶紧给我过来！\n```\n')] 

输出：
```
确实，此刻我情绪异常激动，我的搅拌机的盖子竟然意外飞出，将厨房的墙壁溅满了果汁。更为糟糕的是，保修条款并不涵盖清洁厨房的费用。朋友，我需要你尽快前来协助解决这个问题。
```


In [95]:
service_reply = """嘿，顾客， \
保修不包括厨房的清洁费用， \
因为您在启动搅拌机之前 \
忘记盖上盖子而误用搅拌机, \
这是您的错。 \
倒霉！ 再见！
"""

service_style_pirate = """\
一个有礼貌的语气 \
使用海盗风格\
"""
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply
)

service_response = chat(service_messages)
print(service_response.content)

啊哈，船长！咱今日要来解个谜题，来自咱的顾客朋友。咱得说，这事儿可有点儿搞笑。保修那事儿，它不包含厨房清洁费用哦，这回是咋回事呢？原来呀，您在启动那搅拌机器之前，忘了盖上盖子，结果就有点儿误打误撞地用上了它。哎呀，这可真是个教训，不是吗？别担心，下次记得盖好盖子，这样搅拌机就能好好工作了。

好了，船长，现在咱们该说再见了。希望这次的小插曲不会影响到您在船上的愉快时光。记得，下次使用时要小心谨慎，别让小错误变成大麻烦。再见啦，祝您航海顺利！

*注：这里的“船长”和“航海”是采用了一种更轻松、幽默的海盗风格，以增加趣味性。同时，保持了原文中关于保修条款和注意事项的信息。*


## langchain.output_parsers

### ResponseSchema
ResponseSchema 用于定义输出的字段，name 为字段名，description 为字段描述。
### StructuredOutputParser
- 输入：字段列表
- 输出：输出解析器，即可用于得到格式化输出提示，也可用于对LLM结果的解析。
output_parser.get_format_instructions()得到格式化输出指令。



In [96]:
from langchain.prompts import ChatPromptTemplate
## 不使用输出解析器
## 通过要求以json格式输出来保证输出格式
customer_review = """\
这款吹叶机非常神奇。 它有四个设置：\
吹蜡烛、微风、风城、龙卷风。 \
两天后就到了，正好赶上我妻子的\
周年纪念礼物。 \
我想我的妻子会喜欢它到说不出话来。 \
到目前为止，我是唯一一个使用它的人，而且我一直\
每隔一天早上用它来清理草坪上的叶子。 \
它比其他吹叶机稍微贵一点，\
但我认为它的额外功能是值得的。
"""

review_template = """\
对于以下文本，请从中提取以下信息：

礼物：该商品是作为礼物送给别人的吗？ \
如果是，则回答 是的；如果否或未知，则回答 不是。

交货天数：产品需要多少天\
到达？ 如果没有找到该信息，则输出-1。

价钱：提取有关价值或价格的任何句子，\
并将它们输出为逗号分隔的 Python 列表。

使用以下键将输出格式化为 JSON：
礼物
交货天数
价钱

文本: {text}
"""

prompt_template = ChatPromptTemplate.from_template(review_template)
print("提示模版：", prompt_template)


messages = prompt_template.format_messages(text=customer_review)


chat = ChatOpenAI(model='qwen2-7b-instruct',temperature=0.0)
response = chat(messages)

print("结果类型:", type(response.content))
print("结果:", response.content)


提示模版： input_variables=['text'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='对于以下文本，请从中提取以下信息：\n\n礼物：该商品是作为礼物送给别人的吗？ 如果是，则回答 是的；如果否或未知，则回答 不是。\n\n交货天数：产品需要多少天到达？ 如果没有找到该信息，则输出-1。\n\n价钱：提取有关价值或价格的任何句子，并将它们输出为逗号分隔的 Python 列表。\n\n使用以下键将输出格式化为 JSON：\n礼物\n交货天数\n价钱\n\n文本: {text}\n'))]
结果类型: <class 'str'>
结果: ```json
{
  "礼物": "不是",
  "交货天数": 2,
  "价钱": ["比其他吹叶机稍微贵一点"]
}
```


In [97]:
review_template_2 = """\
对于以下文本，请从中提取以下信息：：

礼物：该商品是作为礼物送给别人的吗？
如果是，则回答 是的；如果否或未知，则回答 不是。

交货天数：产品到达需要多少天？ 如果没有找到该信息，则输出-1。

价钱：提取有关价值或价格的任何句子，并将它们输出为逗号分隔的 Python 列表。

文本: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

gift_schema = ResponseSchema(name="礼物",
                             description="这件物品是作为礼物送给别人的吗？\
                            如果是，则回答 是的，\
                            如果否或未知，则回答 不是。")

delivery_days_schema = ResponseSchema(name="交货天数",
                                      description="产品需要多少天才能到达？\
                                      如果没有找到该信息，则输出-1。")

price_value_schema = ResponseSchema(name="价钱",
                                    description="提取有关价值或价格的任何句子，\
                                    并将它们输出为逗号分隔的 Python 列表")


response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
print("输出格式规定：",format_instructions)


输出格式规定： The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"礼物": string  // 这件物品是作为礼物送给别人的吗？                            如果是，则回答 是的，                            如果否或未知，则回答 不是。
	"交货天数": string  // 产品需要多少天才能到达？                                      如果没有找到该信息，则输出-1。
	"价钱": string  // 提取有关价值或价格的任何句子，                                    并将它们输出为逗号分隔的 Python 列表
}
```


In [98]:
messages = prompt.format_messages(text=customer_review, format_instructions=format_instructions)
print("第一条客户消息:",messages[0].content)


第一条客户消息: 对于以下文本，请从中提取以下信息：：

礼物：该商品是作为礼物送给别人的吗？
如果是，则回答 是的；如果否或未知，则回答 不是。

交货天数：产品到达需要多少天？ 如果没有找到该信息，则输出-1。

价钱：提取有关价值或价格的任何句子，并将它们输出为逗号分隔的 Python 列表。

文本: 这款吹叶机非常神奇。 它有四个设置：吹蜡烛、微风、风城、龙卷风。 两天后就到了，正好赶上我妻子的周年纪念礼物。 我想我的妻子会喜欢它到说不出话来。 到目前为止，我是唯一一个使用它的人，而且我一直每隔一天早上用它来清理草坪上的叶子。 它比其他吹叶机稍微贵一点，但我认为它的额外功能是值得的。


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"礼物": string  // 这件物品是作为礼物送给别人的吗？                            如果是，则回答 是的，                            如果否或未知，则回答 不是。
	"交货天数": string  // 产品需要多少天才能到达？                                      如果没有找到该信息，则输出-1。
	"价钱": string  // 提取有关价值或价格的任何句子，                                    并将它们输出为逗号分隔的 Python 列表
}
```



In [101]:
response = chat(messages)

print("结果类型:", type(response.content))
print("结果:", response.content)

output_dict = output_parser.parse(response.content)

print("解析后的结果类型:", type(output_dict))
print("解析后的结果:", output_dict)

结果类型: <class 'str'>
结果: ```json
{
	"礼物": "是的",
	"交货天数": "两天后",
	"价钱": ["比其他吹叶机稍微贵一点"]
}
```
解析后的结果类型: <class 'dict'>
解析后的结果: {'礼物': '是的', '交货天数': '两天后', '价钱': ['比其他吹叶机稍微贵一点']}
